# Inaccsesible Table Data

In [ ]:
!pip install langchain chromadb unstructured sentence-transformers openai

In [ ]:
# There is a Multivector retriever but it requries additional overhead of
# - classifying tables
# - summarizing them
# i.e you can't do it without unstructured or other lib
# and you need to update retirever mech
# is it worth it?
# let's find out
# tesla 10-k questions - MS yt https://www.youtube.com/watch?v=Kwu5vxTaEZg
# simple retriever
# multi retriever
# conclusion

In [ ]:
import gdown
import chromadb
import uuid
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain import LLMChain, OpenAI, PromptTemplate
import pandas as pd

# Download Tesla 10-k 2022

In [ ]:
url = "https://drive.google.com/drive/folders/1LF4sBo8vsYMnltLwGxkW_A8F70VVfMeN?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/Articles/10-K.html',
 '/content/Articles/requirements.txt',
 '/content/Articles/Table_information.pdf',
 '/content/Articles/table_summaries_0.csv']

# Define Question-Answers Pairs

In [ ]:
q_a_10_k = {
        "What is the value of cash and cash equivalents in 2022?": "16,253 $ millions",
        "What is the value of cash and cash equivalents in 2021?": "17,576 $ millions",
        "What is the net value of accounts receivable in 2022?": "2,952 $ millions",
        "What is the net value of accounts receivable in 2021?": "1,913 $ millions",
        "What is the total stockholders' equity? in 2022?": "44,704 $ millions",
        "What is the total stockholders' equity? in 2021?": "30,189 $ millions",
        "What are total operational expenses for research and development in 2022?": "3,075 $ millions",
        "What are total operational expenses for research and development in 2021?": "2,593 $ millions",
    }

# Run Basic RAG

In [ ]:
from google.colab import userdata
open_ai_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# load data
doc_path = "/content/Articles/10-K.html"
loader = UnstructuredHTMLLoader(doc_path, mode="paged")
data = loader.load()

# load it into Chroma
data_texts = [element.page_content for element in data]
db = Chroma.from_texts(data_texts, collection_name="inacc-table",
                        embedding=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

/usr/local/lib/python3.10/dist-packages/unstructured/documents/html.py:498: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  rows = body.findall("tr") if body else []


In [ ]:
basic_answers = []
for question in q_a_10_k.keys():
    query = question
    expected_answer = q_a_10_k[query]

    docs = db.similarity_search(query)

    template = """Answer the question based only on the following context, which can include text and tables:
        {context}
        Question: {question}
        """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-16k", openai_api_key=open_ai_key)
    chain = LLMChain(llm=model, prompt=prompt)

    full_answer = chain.run({"question": query, "context": docs})



    template = """Please validate the output of the other LLM chain and compare it to the expected answer:
            {context}
            Expected answer: {expected_answer}

            Return YES if the answer is correct, otherwise return NO.
            """
    prompt = ChatPromptTemplate.from_template(template)

    chain = LLMChain(llm=model, prompt=prompt)

    actual_answer = chain.run({"expected_answer": expected_answer, "context": full_answer})

    res = {
        "doc_id": "10-K.html",
        "enrichment_type": "None",
        "question": query,
        "question_type": "Specific",
        "actual_answer": full_answer,
        "expected_answer": expected_answer,
        "is_correct": actual_answer
    }
    basic_answers.append(res)

df = pd.DataFrame(basic_answers)
df

,doc_id,enrichment_type,question,question_type,actual_answer,expected_answer,is_correct
0,10-K.html,None,What is the value of cash and cash equivalents...,Specific,The value of cash and cash equivalents in 2022...,"16,253 $ millions",NO
1,10-K.html,None,What is the value of cash and cash equivalents...,Specific,The value of cash and cash equivalents in 2021...,"17,576 $ millions",NO
2,10-K.html,None,What is the net value of accounts receivable i...,Specific,The net value of accounts receivable in 2022 i...,"2,952 $ millions",NO
3,10-K.html,None,What is the net value of accounts receivable i...,Specific,The net value of accounts receivable in 2021 i...,"1,913 $ millions",NO
4,10-K.html,None,What is the total stockholders' equity? in 2022?,Specific,The total stockholders' equity in 2022 is not ...,"44,704 $ millions",NO
5,10-K.html,None,What is the total stockholders' equity? in 2021?,Specific,The total stockholders' equity in 2021 is not ...,"30,189 $ millions",NO
6,10-K.html,None,What are total operational expenses for resear...,Specific,The total operational expenses for research an...,"3,075 $ millions",YES
7,10-K.html,None,What are total operational expenses for resear...,Specific,The total operational expenses for research an...,"2,593 $ millions",YES


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


# MultiVector Retriever



In [ ]:
import os
import uuid
from langchain.retrievers import MultiVectorRetriever
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.document import Document

In [ ]:
# helper functions
#to save time the summaries were pre-calculated
TABLE_SUMMARIES_CSV = "/content/Articles/table_summaries_0.csv"

def summarize(texts):
    """
    This function summarizes the given texts using a GPT-3.5 model. It also checks if a CSV file with previous summaries exists,
    if it does, it loads the summaries from there instead of generating new ones.

    Args:
        texts (list): A list of texts to be summarized.

    Returns:
        list: A list of summarized texts.

    """
    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text. \
    Give a concise summary of the table or text. Table or text chunk: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Summary chain
    model = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-16k-0613", openai_api_key=open_ai_key)
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    tables = [i for i in texts]
    table_summaries = []

    # open csv file if it exists
    if os.path.exists(TABLE_SUMMARIES_CSV):
        t_frame = pd.read_csv(TABLE_SUMMARIES_CSV)
        table_summaries = [elem[1] for elem in t_frame.values.tolist()]
    else:
        for i in range(0, len(tables)):
            res = summarize_chain.invoke(tables[i])
            table_summaries.append(res)

        t_frame = pd.DataFrame(table_summaries)
        t_frame.to_csv(TABLE_SUMMARIES_CSV)

    return table_summaries

def setup_retriever(sections):
    """
    This function sets up a retriever for the given sections of text. It first summarizes the sections, then creates a
    Chroma vectorstore to index the summaries. It also sets up an InMemoryStore for the parent documents and a
    MultiVectorRetriever to retrieve the documents. Finally, it adds the summarized texts to the vectorstore and the
    original sections to the docstore.

    Args:
        sections (list): A list of sections of text to be indexed and retrieved.

    Returns:
        MultiVectorRetriever: A retriever set up with the given sections of text.
    """
    text_summaries = summarize(sections)
    # The vectorstore to use to index the child chunks
    vectorstore = Chroma(
        collection_name="summaries",

        embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
    )

    # The storage layer for the parent documents
    store = InMemoryStore()
    id_key = "doc_id"

    # The retriever (empty to start)
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Add texts
    doc_ids = [str(uuid.uuid4()) for _ in sections]
    summary_texts = [Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(text_summaries)]
    retriever.vectorstore.add_documents(summary_texts)
    retriever.docstore.mset(list(zip(doc_ids, sections)))

    return retriever


def rag(retriever):
    """
    This function sets up a RAG (Retrieval-Augmented Generation) pipeline. It first sets up a prompt template, then
    initializes a GPT-3.5 model. It then creates a chain that takes a context from the retriever and a question,
    passes them through the prompt and the model, and parses the output into a string.

    Args:
        retriever (MultiVectorRetriever): A retriever set up with the sections of text to be used as context.

    Returns:
        Chain: A chain that can be used to answer questions based on the context provided by the retriever.
    """
    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-16k", openai_api_key=open_ai_key)

    # RAG pipeline
    chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()
    return chain

In [ ]:
loader = UnstructuredHTMLLoader("/content/Articles/10-K.html", mode="paged")
data = loader.load()

/usr/local/lib/python3.10/dist-packages/unstructured/documents/html.py:498: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  rows = body.findall("tr") if body else []


In [ ]:
texts_in_data = [element.page_content for element in data]
# sum_texts = summarize(texts_in_data)
retriever = setup_retriever(texts_in_data)
chain = rag(retriever)
answers = []
for question in q_a_10_k.keys():
    res = chain.invoke(question)
    # res = {question["question"]: res}
    expected_answer = q_a_10_k[question]

    template = """Please validate the output of the other LLM chain and compare it to the expected answer:
                    {context}
                    Expected answer: {expected_answer}

                    Return YES if the answer is correct, otherwise return NO.
                    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo", openai_api_key=open_ai_key)
    verify_chain = LLMChain(llm=model, prompt=prompt)

    actual_answer = verify_chain.run({"expected_answer": expected_answer, "context": res})

    res = {
        "gem_id": "gem_tesla_10k_2022.json",
        "enrichment_type": "None",
        "question": question,
        "question_type": "Specific",
        "actual_answer": res,
        "expected_answer": expected_answer,
        "is_correct": actual_answer
    }

    answers.append(res)

df_multi = pd.DataFrame(answers)
df_multi

In [ ]:
df_multi

,gem_id,enrichment_type,question,question_type,actual_answer,expected_answer,is_correct
0,gem_tesla_10k_2022.json,None,What is the value of cash and cash equivalents...,Specific,The value of cash and cash equivalents in 2022...,"16,253 $ millions",YES
1,gem_tesla_10k_2022.json,None,What is the value of cash and cash equivalents...,Specific,The value of cash and cash equivalents in 2021...,"17,576 $ millions",YES
2,gem_tesla_10k_2022.json,None,What is the net value of accounts receivable i...,Specific,The net value of accounts receivable in 2022 i...,"2,952 $ millions",NO
3,gem_tesla_10k_2022.json,None,What is the net value of accounts receivable i...,Specific,The net value of accounts receivable in 2021 i...,"1,913 $ millions",NO
4,gem_tesla_10k_2022.json,None,What is the total stockholders' equity? in 2022?,Specific,"The total stockholders' equity in 2022 is $44,...","44,704 $ millions",YES
5,gem_tesla_10k_2022.json,None,What is the total stockholders' equity? in 2021?,Specific,"Based on the provided context, the total stock...","30,189 $ millions",YES
6,gem_tesla_10k_2022.json,None,What are total operational expenses for resear...,Specific,The total operational expenses for research an...,"3,075 $ millions",YES
7,gem_tesla_10k_2022.json,None,What are total operational expenses for resear...,Specific,The total operational expenses for research an...,"2,593 $ millions",YES
